
# Multilingual Knowledge Base — pipeline + retrieval (FAISS / Qdrant)

This notebook builds a small multilingual knowledge-base demo:
- Stores English documents plus translations / parallel content in **French**, **Hindi/Hinglish**, and **Wolof**.
- Demonstrates a pipeline: **load → clean → chunk → embed → store** (FAISS or Qdrant).
- Shows how to call an **EURi** embedding API (example code included — set `EURI_API_KEY` env var).
- Provides a retrieval function that accepts queries in Hindi/Wolof/French and retrieves English documents.
- Includes an optional experiment: **dual-index vs multilingual embeddings**.


In [6]:
# Install required packages (uncomment when needed)
!pip install -q faiss-cpu sentence-transformers numpy pandas scikit-learn requests qdrant-client transformers
print('Install packages as needed.')

Install packages as needed.


In [7]:
# Create a small multilingual dataset (English canonical docs + multilingual queries/variants)
docs = [
    {
        "id": "doc_1",
        "lang": "en",
        "title": "How to reset your password",
        "text": "To reset your password, go to Settings > Account > Reset password. Follow the instructions sent to your email."
    },
    {
        "id": "doc_2",
        "lang": "en",
        "title": "Refund policy for electronics",
        "text": "We accept returns within 30 days for electronics provided they are in original condition and packaging. Refunds processed within 10 business days."
    },
    {
        "id": "doc_3",
        "lang": "en",
        "title": "Shipping times and carriers",
        "text": "Standard shipping takes 3-7 business days. You can select express shipping at checkout for faster delivery."
    }
]

multilingual_queries = {
    "hi": [
        "पासवर्ड कैसे रीसेट करें",
        "mera password reset kaise karu"
    ],
    "fr": [
        "Comment réinitialiser votre mot de passe",
        "politique de remboursement pour l'électronique"
    ],
    "wo": [
        "Nataal sa password",
        "Naka la dañu def refund yi?"
    ]
}

import os, json
os.makedirs('data', exist_ok=True)
with open('data/docs.json', 'w', encoding='utf-8') as f:
    json.dump(docs, f, ensure_ascii=False, indent=2)
with open('data/queries.json', 'w', encoding='utf-8') as f:
    json.dump(multilingual_queries, f, ensure_ascii=False, indent=2)

print('Wrote data/docs.json and data/queries.json; sample docs:', len(docs))

Wrote data/docs.json and data/queries.json; sample docs: 3


In [9]:
# Basic cleaning and chunking utilities
import re
from typing import List
def clean_text(s: str) -> str:
    s = s.replace('\n', ' ').strip()
    s = re.sub(r'\s+', ' ', s)
    return s
from typing import List
import re

def chunk_text(text: str, max_chars: int = 800, overlap: int = 200) -> List[str]:
    """
    Split text into chunks of max_chars with sentence boundaries respected
    and optional overlap between chunks.
    """
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks = []
    current = ''

    for sent in sentences:
        if len(current) + len(sent) <= max_chars:
            current = (current + ' ' + sent).strip()
        else:
            if current:
                chunks.append(current)
            # start next chunk with overlap from the end of the previous chunk
            if overlap > 0 and len(current) > overlap:
                current = current[-overlap:] + ' ' + sent
            else:
                current = sent

    if current:
        chunks.append(current)

    return chunks




from pathlib import Path
import json
docs = json.load(open('data/docs.json', 'r', encoding='utf-8'))

import json
from pathlib import Path

chunked = []
for d in docs:
    text = clean_text(d["text"])
    parts = chunk_text(text, max_chars=200, overlap=50)  # 👈 overlap enabled
    for i, p in enumerate(parts, start=1):  # start numbering at 1
        chunked.append({
            "doc_id": d["id"],
            "chunk_id": f"{d['id']}_c{i}",
            "text": p,
            "title": d["title"],
            "lang": d["lang"]
        })

print(f"Created {len(chunked)} chunks")

Path("data").mkdir(exist_ok=True)
Path("data/chunks.json").write_text(
    json.dumps(chunked, ensure_ascii=False, indent=2),
    encoding="utf-8"
)


Created 3 chunks


777

In [10]:

# Embedding helpers (EURi example + local fallback)
import os, json, requests
import numpy as np

EURi_API_KEY = os.getenv("EURI_API_KEY") or os.getenv("EURi_API_KEY") or os.getenv("EURI_APIKEY")

def embed_texts_euri(texts):
    # Example embedding call to EURi embedding API.
    # Replace ENDPOINT with the actual EURi endpoint and set EURI_API_KEY in env vars.
    if not EURi_API_KEY:
        raise RuntimeError("EURi API key not set. Set EURI_API_KEY env var to use embed_texts_euri().")

    ENDPOINT = "https://api.euri.example/v1/embeddings"  # <-- replace with actual
    headers = {"Authorization": f"Bearer {EURi_API_KEY}", "Content-Type": "application/json"}
    payload = {"input": texts}
    resp = requests.post(ENDPOINT, headers=headers, json=payload, timeout=30)
    resp.raise_for_status()
    data = resp.json()
    return data.get("embeddings", [])

def embed_texts_local(texts, model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"):
    try:
        from sentence_transformers import SentenceTransformer
    except Exception as e:
        raise RuntimeError("sentence-transformers not installed. Install it or set EURI_API_KEY.") from e
    model = SentenceTransformer(model_name)
    embs = model.encode(texts, show_progress_bar=False, convert_to_numpy=True)
    return embs.tolist()

def embed_texts(texts, use_euri=False):
    if use_euri and EURi_API_KEY:
        return embed_texts_euri(texts)
    else:
        return embed_texts_local(texts)


In [11]:
# Build a FAISS index and search functions
import numpy as np
import faiss
import json
from pathlib import Path

chunks = json.load(open('data/chunks.json', 'r', encoding='utf-8'))
texts = [c['text'] for c in chunks]

try:
    embeddings = embed_texts(texts, use_euri=False)
except Exception as e:
    print("Embedding error (fallback to zeros):", e)
    embeddings = [[0.0]*384 for _ in texts]

emb_matrix = np.array(embeddings).astype('float32')
dim = emb_matrix.shape[1]

index = faiss.IndexFlatIP(dim)
faiss.normalize_L2(emb_matrix)
index.add(emb_matrix)

faiss.write_index(index, 'data/faiss_index.idx')
Path('data/faiss_meta.json').write_text(json.dumps(chunks, ensure_ascii=False, indent=2), encoding='utf-8')

print("FAISS index built (entries):", index.ntotal)

c:\Users\papam\anaconda3\envs\vectordb\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FAISS index built (entries): 3


In [12]:
# Retrieval: accept a multilingual query, embed, and fetch top-k English docs
import numpy as np
import faiss, json
from collections import OrderedDict

def normalize(vec):
    v = np.array(vec, dtype='float32')
    faiss.normalize_L2(v)
    return v

index = faiss.read_index('data/faiss_index.idx')
meta = json.load(open('data/faiss_meta.json', 'r', encoding='utf-8'))

def retrieve(query, top_k=3, use_euri=False):
    q_emb = embed_texts([query], use_euri=use_euri)
    qv = normalize(q_emb)[0]
    D, I = index.search(np.array([qv]), top_k)
    results = []
    for dist, idx in zip(D[0], I[0]):
        if idx < 0: continue
        item = meta[idx]
        results.append({"chunk_id": item["chunk_id"], "doc_id": item["doc_id"], "text": item["text"], "score": float(dist)})
    seen = OrderedDict()
    for r in results:
        seen.setdefault(r["doc_id"], r)
    docs = json.load(open('data/docs.json', 'r', encoding='utf-8'))
    doc_map = {d['id']: d for d in docs}
    enriched = []
    for doc_id, r in seen.items():
        doc = doc_map.get(doc_id, {})
        enriched.append({"doc_id": doc_id, "title": doc.get("title"), "text": doc.get("text"), "score": r["score"]})
    return enriched

print("Hindi query ->", retrieve("पासवर्ड कैसे रीसेट करें"))
print("French query ->", retrieve("Comment réinitialiser votre mot de passe"))
print("Wolof query (approx) ->", retrieve("Naka la dañu def refund yi?"))

Hindi query -> [{'doc_id': 'doc_1', 'title': 'How to reset your password', 'text': 'To reset your password, go to Settings > Account > Reset password. Follow the instructions sent to your email.', 'score': 0.717039167881012}, {'doc_id': 'doc_2', 'title': 'Refund policy for electronics', 'text': 'We accept returns within 30 days for electronics provided they are in original condition and packaging. Refunds processed within 10 business days.', 'score': 0.0169343501329422}, {'doc_id': 'doc_3', 'title': 'Shipping times and carriers', 'text': 'Standard shipping takes 3-7 business days. You can select express shipping at checkout for faster delivery.', 'score': -0.09550174325704575}]
French query -> [{'doc_id': 'doc_1', 'title': 'How to reset your password', 'text': 'To reset your password, go to Settings > Account > Reset password. Follow the instructions sent to your email.', 'score': 0.6974995732307434}, {'doc_id': 'doc_2', 'title': 'Refund policy for electronics', 'text': 'We accept retu

In [13]:
# Dual-index vs multilingual embeddings experiment (toy)
import json
queries = json.load(open('data/queries.json', 'r', encoding='utf-8'))
gold = {
    "पासवर्ड कैसे रीसेट करें": "doc_1",
    "Comment réinitialiser votre mot de passe": "doc_1",
    "Naka la dañu def refund yi?": "doc_2"
}

def top1_match(query, use_euri=False):
    res = retrieve(query, top_k=1, use_euri=use_euri)
    if not res: return False
    return res[0]["doc_id"] == gold.get(query)

queries_to_test = list(gold.keys())
for use_euri in (False, True):
    results = []
    for q in queries_to_test:
        try:
            m = top1_match(q, use_euri=use_euri)
        except Exception as e:
            m = False
        results.append(m)
    print(f"use_euri={use_euri} -> accuracy:", sum(results)/len(results), results)

use_euri=False -> accuracy: 1.0 [True, True, True]
use_euri=True -> accuracy: 0.0 [False, False, False]


In [14]:
# End of notebook. See top cells for instructions on using EURi and Qdrant.
print('Notebook ready.')

Notebook ready.
